## 설정 초기화

In [3]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        database='cnu_data',
        user='root',
        password='dydrkfl#7!'
    )
    
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"MySQL 서버에 연결되었습니다. 버전: {db_info}")
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print(f"현재 데이터베이스: {record[0]}")

except Error as e:
    print(f"MySQL 연결 중 에러 발생: {e}")


MySQL 서버에 연결되었습니다. 버전: 9.3.0
현재 데이터베이스: cnu_data


/var/folders/kv/jv4ntr9x5h3f_0rbw1xytqbm0000gp/T/ipykernel_55958/499930515.py:14: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()


In [ ]:
import requests
import json
import datetime 
def log_api_error(endpoint, error_message):
    """
    API 호출 에러를 로그 파일에 기록하는 함수
    
    Args:
        endpoint (str): API 엔드포인트 URL
        error_message (str): 발생한 에러 메시지
    """
    try:
        with open('api_log.log', 'a', encoding='utf-8') as log_file:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_entry = f"[{timestamp}] API 호출 실패\n"
            log_entry += f"엔드포인트: {endpoint}\n"
            log_entry += f"에러 메시지: {error_message}\n"
            log_entry += "-" * 80 + "\n"
            log_file.write(log_entry)
    except Exception as e:
        print(f"로그 파일 작성 중 에러 발생: {e}")

def fetch_data(endpoint):
    """
    CNU API에서 데이터를 가져오는 함수
    
    Args:
        endpoint (str): API 엔드포인트 URL
        
    Returns:
        dict: API 응답 데이터
    """
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    try:
        response = requests.post(endpoint, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        print(f"API 응답 상태: {response.status_code}")
        print(f"응답 데이터: {json.dumps(data, indent=2, ensure_ascii=False)}")
        
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 에러 발생: {e}")
        
        # 에러 발생 시 진행상황 저장하기(로그)
        log_api_error(endpoint, str(e))
        return None

In [ ]:
AUTH_KEY = '081E3B5B5C5E47409E3AB0BE3FBFC9FDC7798045'
CNU_API_ENDPOINT = 'https://api.cnu.ac.kr/svc/offcam/pub'

# TODO: 각 데이터 테이블 정의하고, 컬럼 정의
# TODO: API 호출 시 페이지 수 정하고, 남은 페이지 카운트 파악해서 진행상황에 기록하기
# TODO: 수강신청 데이터 전처리 전략 세우기(수강신청을 얼마나 많이 했는지 등)
# TODO: 저장된 데이터와 데이터베이스를 유지한 채 export할 방법 찾기

### 강의계획서

In [ ]:
p_yr = 2023
p_open_shtm = 1
page = 1
endpoint = f'{CNU_API_ENDPOINT}/lsnSmry?AUTH_KEY={AUTH_KEY}&P_YR={p_yr}&P_OPEN_SHTM ={p_open_shtm}&page={page}'

# API 호출
data = fetch_data(endpoint)


### CNU 게시판 목록

In [ ]:
endpoint = f'{CNU_API_ENDPOINT}/cmsBoard?AUTH_KEY={AUTH_KEY}'

# API 호출
data = fetch_data(endpoint)
board_list = data['result']['list'] # TODO: 올바른 데이터로 변환 필요

### CNU 게시판 내용

In [ ]:
p_board_no = 1
endpoint = f'{CNU_API_ENDPOINT}/homepageboardContents?AUTH_KEY={AUTH_KEY}&P_board_no={p_board_no}'

# API 호출
data = fetch_data(endpoint)

### 수강신청

In [ ]:
p_yr = 2024
page = 1
endpoint = f'{CNU_API_ENDPOINT}/SugangInfo?AUTH_KEY={AUTH_KEY}&P_OPEN_YR={p_yr}&page={page}'

# API 호출
data = fetch_data(endpoint)